In [499]:
#!pip install dgl -f https://data.dgl.ai/wheels/repo.html

#!pip install dglgo -f https://data.dgl.ai/wheels-test/repo.html

In [500]:
%matplotlib inline
import os

os.environ["DGLBACKEND"] = "pytorch"
import dgl
import numpy as np
import networkx as nx
import torch
import torch.nn as nn
import dgl.function as fn

# My Graph

In [501]:
f = dgl.graph(([0, 1, 2, 3], [1, 2, 3, 4]))

In [502]:
g = dgl.heterograph({('Id', 'follower', 'Id'): (torch.tensor([1, 2, 3, 0]), torch.tensor([1, 2, 3, 0]))})

# Computing with DGLGraph

DGLGraph.apply_nodes

In [503]:
f.ndata['h'] = torch.ones(5, 3)

In [504]:
f.apply_nodes(lambda nodes: {'x' : nodes.data['h'] * 3})
f.ndata['x']

tensor([[3., 3., 3.],
        [3., 3., 3.],
        [3., 3., 3.],
        [3., 3., 3.],
        [3., 3., 3.]])

In [505]:
g.nodes['Id'].data['h'] = torch.ones(4, 4)
g.apply_nodes(lambda nodes: {'h': nodes.data['h'] * 5}, ntype='Id')
g.nodes['Id'].data['h']

tensor([[5., 5., 5., 5.],
        [5., 5., 5., 5.],
        [5., 5., 5., 5.],
        [5., 5., 5., 5.]])

DGLGraph.apply_edges

In [506]:
f.ndata['h'] = torch.ones(5, 2)
f.apply_edges(lambda edges: {'x' : edges.src['h'] + edges.dst['h']})
f.edata['x']

tensor([[2., 2.],
        [2., 2.],
        [2., 2.],
        [2., 2.]])

In [507]:
f.apply_edges(fn.u_add_v('h', 'h', 'x'))
f.edata['x']

tensor([[2., 2.],
        [2., 2.],
        [2., 2.],
        [2., 2.]])

In [508]:
g.edges[('Id', 'follower', 'Id')].data['h'] = torch.ones(4, 2)
g.apply_edges(lambda edges: {'h': edges.data['h'] * 2})
g.edges[('Id', 'follower', 'Id')].data['h']

tensor([[2., 2.],
        [2., 2.],
        [2., 2.],
        [2., 2.]])

DGLGraph.send_and_recv

In [509]:
f.ndata['x'] = torch.ones(5, 2)

In [510]:
f.send_and_recv(([1, 2], [2, 3]), fn.copy_u('x', 'm'), fn.sum('m', 'h'))
f.ndata['h']

tensor([[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]])

In [511]:
f.send_and_recv([0, 2, 3], fn.copy_u('x', 'm'), fn.sum('m', 'h'))
f.ndata['h']

tensor([[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]])

In [512]:
g.nodes['Id'].data['h'] = torch.tensor([[0.], [1.], [2.], [3]])
g.send_and_recv(g['follower'].edges(), fn.copy_u('h', 'm'),
                fn.sum('m', 'h'), etype='follower')
g.nodes['Id'].data['h']

tensor([[0.],
        [1.],
        [2.],
        [3.]])

In [513]:
# Define the function for sending node features as messages.
def send_source(edges):
    return {'m': edges.src['x']}
# Sum the messages received and use this to replace the original node feature.
def simple_reduce(nodes):
    return {'x': nodes.mailbox['m'].sum(1)}

In [514]:
f.send_and_recv(f.edges(), send_source, simple_reduce)
f.ndata['x']

tensor([[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]])

DGLGraph.pull

In [515]:
f.ndata['x'] = torch.ones(5, 2)
f.pull([0, 3, 4], fn.copy_u('x', 'm'), fn.sum('m', 'h'))
f.ndata['h']

tensor([[0., 0.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]])

In [516]:
#g.nodes['Id'].data['h'] = torch.tensor([[0.], [1.], [2.], [3.]])
g['follower'].pull(2, fn.copy_u('h', 'm'), fn.sum('m', 'h'), etype='follower')
g.nodes['Id'].data['h']

tensor([[0.],
        [1.],
        [2.],
        [3.]])

DGLGraph.push

In [517]:
f.ndata['x'] = torch.ones(5, 2)
f.push([0, 1], fn.copy_u('x', 'm'), fn.sum('m', 'h'))
f.ndata['h']

tensor([[0., 0.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]])

In [518]:
#g.nodes['Id'].data['h'] = torch.tensor([[0.], [1.], [2.], [3.]])
g['follower'].push(0, fn.copy_u('h', 'm'), fn.sum('m', 'h'), etype='follower')
g.nodes['Id'].data['h']

tensor([[0.],
        [1.],
        [2.],
        [3.]])

DGLGraph.update_all

In [519]:
f.update_all(fn.copy_u('x', 'm'), fn.sum('m', 'h'))
f.ndata['h']

tensor([[0., 0.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]])

In [520]:
g['follower'].update_all(fn.copy_u('h', 'm'), fn.sum('m', 'h'), etype='follower')
g.nodes['Id'].data['h']

tensor([[0.],
        [1.],
        [2.],
        [3.]])

In [521]:
g.nodes['Id'].data['h'] = torch.tensor([[0.], [1.], [2.], [3.]])
g.update_all(fn.copy_u('h', 'm'), fn.sum('m', 'h'))
g.nodes['Id'].data['h']

tensor([[0.],
        [1.],
        [2.],
        [3.]])

# dgl.function

In [522]:
f.ndata['h']

tensor([[0., 0.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]])

In [523]:
f.edata['x']

tensor([[2., 2.],
        [2., 2.],
        [2., 2.],
        [2., 2.]])

In [524]:
g.ndata['h']

tensor([[0.],
        [1.],
        [2.],
        [3.]])

In [525]:
g.edata['h']

tensor([[2., 2.],
        [2., 2.],
        [2., 2.],
        [2., 2.]])

In [526]:
f1 = f
g1 = g

copy_u


In [527]:
def message_func(edges):
    return {'m': edges.src['h']}

In [528]:
f.update_all(message_func, fn.sum('m', 'h'))

In [529]:
f.ndata['h']

tensor([[0., 0.],
        [0., 0.],
        [1., 1.],
        [1., 1.],
        [1., 1.]])

In [530]:
g.update_all(message_func, fn.sum('m', 'h'))

In [531]:
g.ndata['h']

tensor([[0.],
        [1.],
        [2.],
        [3.]])

copy_e

In [532]:
f1 = f

In [533]:
def message_func(edges):
    return {'m': edges.data['x']}

In [534]:
f.update_all(message_func, fn.sum('m', 'x'))

In [535]:
f.ndata['x']

tensor([[0., 0.],
        [2., 2.],
        [2., 2.],
        [2., 2.],
        [2., 2.]])

u_add_v

In [536]:
f1 = f

In [537]:
f.update_all(fn.u_add_v('h', 'x', 'm'), fn.sum('m', 'h_new'))
f.ndata['h_new']

tensor([[0., 0.],
        [2., 2.],
        [2., 2.],
        [3., 3.],
        [3., 3.]])

In [538]:
g.update_all(fn.u_add_v('h', 'h', 'm'), fn.sum('m', 'g_new'))
g.ndata['g_new']

tensor([[0.],
        [2.],
        [4.],
        [6.]])

u_mul_v


In [539]:
f = f1

In [540]:
f.update_all(fn.u_mul_v('h', 'x', 'm'), fn.sum('m', 'h_new'))
f.ndata['h_new']

tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        [2., 2.],
        [2., 2.]])

In [541]:
g = g1

In [542]:
g.update_all(fn.u_mul_v('h', 'h', 'm'), fn.sum('m', 'g_new'))
g.ndata['g_new']

tensor([[0.],
        [1.],
        [4.],
        [9.]])

v_add_e


In [543]:
f = f1

In [544]:
f.update_all(fn.v_add_e('h', 'x', 'm'), fn.sum('m', 'h_new'))
f.ndata['h_new']

tensor([[0., 0.],
        [2., 2.],
        [3., 3.],
        [3., 3.],
        [3., 3.]])

In [545]:
g = g1

In [546]:
g.update_all(fn.v_add_e('h', 'h', 'm'), fn.sum('m', 'g_new'))
g.ndata['g_new']

tensor([[2., 2.],
        [3., 3.],
        [4., 4.],
        [5., 5.]])

v_mul_e

In [547]:
f = f1

In [548]:
f.update_all(fn.v_mul_e('h', 'x', 'm'), fn.sum('m', 'h_new'))
f.ndata['h_new']

tensor([[0., 0.],
        [0., 0.],
        [2., 2.],
        [2., 2.],
        [2., 2.]])

In [549]:
g = g1

In [550]:
g.update_all(fn.v_mul_e('h', 'h', 'm'), fn.sum('m', 'g_new'))
g.ndata['g_new']

tensor([[0., 0.],
        [2., 2.],
        [4., 4.],
        [6., 6.]])

v_add_u

In [551]:
f = f1

In [552]:
f.update_all(fn.v_add_u('h', 'x', 'm'), fn.sum('m', 'h_new'))
f.ndata['h_new']

tensor([[0., 0.],
        [0., 0.],
        [3., 3.],
        [3., 3.],
        [3., 3.]])

In [553]:
g = g1

In [554]:
g.update_all(fn.v_add_u('h', 'h', 'm'), fn.sum('m', 'g_new'))
g.ndata['g_new']

tensor([[0.],
        [2.],
        [4.],
        [6.]])

v_mul_u

In [555]:
f = f1

In [556]:
f.update_all(fn.v_mul_u('h', 'x', 'm'), fn.sum('m', 'h_new'))
f.ndata['h_new']

tensor([[0., 0.],
        [0., 0.],
        [2., 2.],
        [2., 2.],
        [2., 2.]])

In [557]:
g = g1

In [558]:
g.update_all(fn.v_mul_u('h', 'h', 'm'), fn.sum('m', 'g_new'))
g.ndata['g_new']

tensor([[0.],
        [1.],
        [4.],
        [9.]])

u_add_e

In [559]:
f = f1

In [560]:
f.update_all(fn.u_add_e('h', 'x', 'm'), fn.sum('m', 'h_new'))
f.ndata['h_new']

tensor([[0., 0.],
        [2., 2.],
        [2., 2.],
        [3., 3.],
        [3., 3.]])

In [561]:
g = g1

In [562]:
g.update_all(fn.u_add_e('h', 'h', 'm'), fn.sum('m', 'g_new'))
g.ndata['g_new']

tensor([[2., 2.],
        [3., 3.],
        [4., 4.],
        [5., 5.]])

u_mul_e

In [563]:
f = f1

In [564]:
f.update_all(fn.u_mul_e('h', 'x', 'm'), fn.sum('m', 'h_new'))
f.ndata['h_new']

tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        [2., 2.],
        [2., 2.]])

In [565]:
g = g1

In [566]:
g.update_all(fn.u_mul_e('h', 'h', 'm'), fn.sum('m', 'g_new'))
g.ndata['g_new']

tensor([[0., 0.],
        [2., 2.],
        [4., 4.],
        [6., 6.]])